In [1]:
!rm -f /content/main.zip
!wget https://raw.githubusercontent.com/Ostap-Kucher/Labs/main/main.zip

'rm' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!unzip /content/main.zip

Archive:  /content/main.zip
   creating: lab1-main/
  inflating: __MACOSX/._lab1-main    
  inflating: lab1-main/.DS_Store     
  inflating: __MACOSX/lab1-main/._.DS_Store  
  inflating: lab1-main/eng texts.txt  
  inflating: __MACOSX/lab1-main/._eng texts.txt  
  inflating: lab1-main/ukr texts.txt  
  inflating: __MACOSX/lab1-main/._ukr texts.txt  
  inflating: lab1-main/README.md     
  inflating: __MACOSX/lab1-main/._README.md  


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer  # імпортуємо токенізатор для перетворення тексту в числа
from tensorflow.keras.utils import to_categorical, pad_sequences  # для перетворення міток та вирівнювання послідовностей
from tensorflow.keras.models import Sequential  # для створення послідовної моделі нейронної мережі
from tensorflow.keras.layers import Embedding, LSTM, Dense  # імпортуємо шари: Embedding, LSTM, Dense
import numpy as np  # імпортуємо numpy для роботи з масивами чисел

# Перевірка версії TensorFlow
print("TensorFlow version:", tf.__version__)

# 2. Підключення файлів з текстами
with open("/content/lab1-main/eng texts.txt", "r", encoding="latin-1") as f:
    eng_texts = f.read().split("\n")  # читаємо всі рядки та розбиваємо на список рядків

with open("/content/lab1-main/ukr texts.txt", "r", encoding="mac_cyrillic") as f:
    ukr_texts = f.read().split("\n")

# Об'єднуємо всі тексти для одного корпусу
texts = eng_texts + ukr_texts

# 3. Очищення тексту (базове: нижній регістр, видалення зайвих пробілів)
cleaned_texts = [t.lower().strip() for t in texts]  # перетворюємо кожен рядок у нижній регістр і прибираємо зайві пробіли
print("Приклад до очищення:", texts[0])
print("Приклад після очищення:", cleaned_texts[0])

# 4. Базова статистика
total_chars = sum(len(t) for t in cleaned_texts)  # обчислюємо загальну кількість символів у всіх рядках
total_sentences = len(cleaned_texts)  # обчислюємо кількість рядків (речень)
print("Обсяг символів у корпусі:", total_chars)
print("Кількість речень:", total_sentences)

# 5. Токенізація (word-level)
max_tokens = 1000  # максимальний розмір словника
tokenizer = Tokenizer(num_words=max_tokens, oov_token="<OOV>")  # створюємо токенізатор з обмеженням словника та OOV токеном для незнайомих слів
tokenizer.fit_on_texts(cleaned_texts)  # навчаємо токенізатор на всіх рядках тексту
sequences = tokenizer.texts_to_sequences(cleaned_texts)  # перетворюємо текст у послідовності чисел (кожне слово-число)
word_index = tokenizer.word_index  # отримуємо словник: слово-номер
print("Розмір словника:", min(len(word_index), max_tokens))

# 6. Формування зразків для моделі
seq_len = 5  # довжина вхідного контексту (скільки слів беремо для передбачення наступного)
X, y = [], []  # створюємо порожні списки для вхідних та вихідних даних
for seq in sequences:  # проходимо по кожній послідовності слів
    for i in range(seq_len, len(seq)):  # починаємо з позиції seq_len до кінця послідовності
        X.append(seq[i-seq_len:i])  # беремо попередні seq_len слів як вхід
        y.append(seq[i])  # беремо наступне слово як мітку
X = np.array(X)  # перетворюємо список X у numpy масив
y = to_categorical(y, num_classes=min(len(word_index)+1, max_tokens))

# 7. Архітектура моделі
vocab_size = min(len(word_index)+1, max_tokens)  # розмір словника для Embedding та Dense
embedding_dim = 50  # розмірність векторного подання слова

model = Sequential([  # створюємо послідовну модель
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_len),  # шар Embedding: слова-вектори
    LSTM(100),  # LSTM шар з 100 нейронами для обробки послідовностей
    Dense(vocab_size, activation='softmax')  # вихідний Dense шар з активацією softmax на всі слова словника
])
model.build(input_shape=(None, seq_len))  # явно створюємо модель з заданою вхідною формою

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # компілюємо модель: крос-ентропія + Adam + точність
model.summary()  # показуємо коротку інформацію про модель

# 8. Навчання моделі
model.fit(X, y, epochs=50, batch_size=16)

# 9. Генерація тексту
def generate_text(seed_text, next_words=10):  # функція для генерації тексту, seed_text — початковий рядок
    result = seed_text.split()  # розбиваємо початковий рядок на список слів
    for _ in range(next_words):  # повторюємо next_words разів
        token_list = tokenizer.texts_to_sequences([" ".join(result[-seq_len:])])[0]  # беремо останні seq_len слів та перетворюємо їх у числа
        token_list = pad_sequences([token_list], maxlen=seq_len)  # вирівнюємо послідовність до довжини seq_len
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]  # передбачаємо наступне слово (номер)
        output_word = tokenizer.index_word.get(predicted, "<OOV>")  # отримуємо слово з номера або <OOV>
        result.append(output_word)  # додаємо передбачене слово до результату
    return " ".join(result)  # об'єднуємо всі слова у рядок і повертаємо

# Приклад генерації тексту англійською
seed = "this is a programme"
generated = generate_text(seed, next_words=10)  # генеруємо 10 слів після початкового рядка
print("Згенерований текст:", generated)

TensorFlow version: 2.19.0
Приклад до очищення: Sleep is a fundamental biological necessity, not a luxury we can easily sacrifice. During the night, your brain remains incredibly active, consolidating memories and processing the day's information. Getting adequate rest is crucial for cognitive function, directly affecting your ability to focus, solve problems, and make decisions. Scientists recommend that most adults aim for seven to nine hours of quality sleep per night. Chronic lack of sleep, or sleep deprivation, can lead to serious health issues, including a weakened immune system. It also increases the risk of developing conditions like diabetes, heart disease, and high blood pressure. Furthermore, poor sleep significantly impacts mood, often leading to irritability, stress, and even depression. To improve sleep hygiene, establish a consistent bedtime routine, even on weekends. Avoid consuming caffeine and large meals close to bedtime, as they can interfere with your natural sleep

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 5, 50)          │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        60,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1000)           │       101,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 211,400 (825.78 KB)

 Trainable params: 211,400 (825.78 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.1704 - loss: 6.4354
Epoch 2/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1872 - loss: 5.4881
Epoch 3/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2008 - loss: 5.2576
Epoch 4/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1909 - loss: 5.1757
Epoch 5/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2039 - loss: 4.9335
Epoch 6/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1996 - loss: 4.9059
Epoch 7/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2020 - loss: 4.7455
Epoch 8/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2136 - loss: 4.6261
Epoch 9/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2200 - loss: 4.5065
Epoch 10/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2190 - loss: 4.4095
Epoch 11/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2307 - loss: 4.3057
Epoch 12/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/ste